## Anomaly Detection using K-means Clustering

In [1]:
#import all the necessary libraries
import sys
import random
import pandas as pd
import numpy as np
from scipy.spatial import distance
import copy
import warnings

#ignore warnings
warnings.filterwarnings("ignore")

In [3]:
#Read the data using standard input(stdin) and put the data into a pandas dataframe
#2D data
dataset = pd.read_csv('t11.dat',header=None)
#1D data
#dataset = pd.read_csv('t10.dat',header=None)
#Illegal data
#dataset = pd.read_csv('t19.dat',header=None)
#to work with any data from command line
#dataset = pd.read_csv(sys.stdin,header=None)

#Find the shape of the dataset.
#Samples are the number of points and feature is the dimension
samples, features = dataset.shape

#Convert the dataframe to an array,data
data = dataset.values.tolist()

In [4]:
'''
Canopy Method

Functionality: This method is used to find the farthest centriods from each other based on their distance.
Step1: A random,initial centroid from the dataset is chosen using the random.choice fucntion
Step2: Using the library function scipy, cdist with metrics as elucidean distance is calculated between initial centroid and all data points.
Step3: Find the maximum distance and the corresponding data point. This becomes the next centroid
Step4: A while loop is run to repeat step3 till the number of centroids determined equals the number of clusters,k. It the centroid gets repeated, the distance and the corresponding
       datapoint is deleted and the next maximum distance and its point is the centroid
Step5: Inside the Step4 while loop, there is another while loop to keep track if no repeating centroids are appendend to the final "centroids" list

Return value: List of centroids whose length equals the number of clusters,k
'''

def canopy(data,features,clu):
    initial_centroid = []   
    i = 1
    distances = 0
    centroids = []
    #select a random centroid from the dataset
    initial_centroid.append(np.reshape(random.choice(data),(1,features)).tolist())

    #find the other centroids based on cluster, clu value
    while i < clu:
            
        data1 = copy.deepcopy(data)

        for j in initial_centroid:
            #sum the distances of all data points to a centroid
            distances += distance.cdist(j,data,metric = "euclidean")
            
        distnce = distances.flatten().tolist()
        max_dist = max(distnce)

        #keep track of repeating centroids
        while [data1[distnce.index(max_dist)]] in initial_centroid:
            del data1[distnce.index(max_dist)]
            del distnce[distnce.index(max_dist)]
            max_dist = max(distnce)

        #append the maximum distance point and next centroid
        initial_centroid.append((np.reshape(data1[distnce.index(max_dist)],(-1,features))).tolist())

        #set distances back to zero for next cluster, k
        distances = 0
        i += 1

    #copy intial_centroids list to centroids list
    centroids = copy.deepcopy(initial_centroid)
    
    #intialise initial_centroid and i back to original values for next cluster iteration
    initial_centroid = []
    i = 1

    return centroids

In [5]:
'''
K-Means assignment function

Functionality: The distance between each datapoint and the a centroid is calculated using scipy.cdist and added as a column to the dataset DF. The minimum distance to a
given point is determined and is assigned to the corresponding cluster.

Return Value: A dataframe with clusters column updated with cluster numbers to all the data points, dictionary with cluster and their points which is used for the
kmeans_update function, original dataframe

'''
def kmeans_assign(centroids,data,dataset,features):
    
    #find the distance of the data points to the each centroids
    for cen in range(len(centroids)):
        
        #create a column in the dataset df with the distances of data points from all the centroids
        #flatten() and tolist() are used to convert the distance numpy array generated and append to the column
        dataset["{}".format(cen)] = distance.cdist(centroids[cen],data,metric = "euclidean").flatten().tolist()
    
    #create the clusters column in the dataset df to assig the cluster
    #using idxmin(axis = 1), minimun distance is determined and corresponding column name is assigned in the cluster column
    dataset["clusters"] = dataset.iloc[:,features:].astype(float).idxmin(axis = 1)

    #create a dictionary of datapoints as values and their cluster as keys
    data_dict = {}

    for clustr in dataset['clusters'].unique():        
        data_dict[clustr] = [[dataset[fea][y] for fea in range(0,features)] for y in dataset[dataset['clusters'] == clustr].index]

    #copy the dataset to a final_df
    final_dataset = copy.deepcopy(dataset)
    #drop the columns for further iterations
    dataset.drop(dataset.iloc[:, features:], axis = 1, inplace = True)
    
    return data_dict,dataset,final_dataset

In [6]:
'''
K-means update function

Functionality: Based on the dataframe returned from the assignment function, the mean of the datapoints for each cluster is calculated and theie centroids for are updated.

Return value: The new updated centroids are returned which is used in the kmeans_assign function to reassign clusters

'''
def kmeans_update(data_dict,features,centroids):

    #create a dictionary with the mean values of the datapoints at cluster level
    avgDict = {}
    
    for k,v in data_dict.items():
        avgDict[k] = np.mean(v, axis = 0)

    #sort the dict to assign the centroids at their corresponding index
    avg_dict = sorted(avgDict.items())

    #update the centroids array with the mean centriod of each cluster
    for ad in range(len(avg_dict)):
        if str(ad) in avg_dict[ad]:
            centroids[ad] = [avg_dict[ad][1]]
 
    return centroids

In [7]:
'''
Silhouette method

Silhouette method is used to find optimum number of clusters for a dataset based on Silhouette Coefficient (SC).

For each point i in the dataset ai, bi, and corresponding si is calculated.

ai is calculated as the mean of the intra cluster distance between a point "i" to all the points in that cluster. 

        ai = (1/|Ci| - 1) * Σ d(i,j)

    where i ≠ j, j ∈ Ci
    Ci = cluster
    |Ci| = length of that cluster
    |Ci| - 1 is done as distance between i and i should not be included.

bi is the mean inter cluster distance of a point "i" to all points in its neignboring clusters. The minimum distance of these bi is the final bi for the point "i".

        bi = min (1 / |Ck|) * Σ d(i,j)

    where j ∈ Ck
    |Ck| = length of Ck cluster

si is the silhouette value for each point "i".

        si = (bi - ai) / max(bi,ai)     if |Ci| > 1
    and
        si = 0 if |Ci| = 1

s(k) is the mean of si over all the points for given number of clusters

Silhouette Coeffecient is the max of sk for the given cluster.

    SC = max(sk)

For a dataset with n points and predefined k clusters, SC is calculated for each k. The optimum k is decided corresponding to max SC.

Return value: The mean si value is returned for a given cluster number.

'''
def silhoutte(data_dict,features):
    
    #declare arrays for ai, bi, si
    ai = []
    max_int = float('inf')
    bi = []
    si = []
    
    for key,value in data_dict.items():
        
            #calculate ai
            for point in data_dict[key]:
                ai_dist = 0
                bi_dist = 0
                
                for sec_point in data_dict[key]:
                    if point != sec_point:                        
                         #intra cluster distance between point "i" to all points
                        ai_dist += distance.cdist([point],[sec_point],metric = "euclidean")
                    else:
                        continue
                    
                if len(data_dict[key]) == 1:
                    mean_ai = [ai_dist]
                else:
                    mean_ai = ai_dist / (len(data_dict[key]) - 1)
                
                ai.append(mean_ai)

                #calculate bi
                for ky,val in data_dict.items():
                    if ky != key:
                        #inter cluster distance between point "i" to all points in its neighboring cluster
                        bi_dist = distance.cdist([point],val,metric = "euclidean")
                    else:
                        continue

                    #mean bi for a cluster
                    cluster_bi_mean = np.mean(bi_dist)

                    #find the min bi between for point "i"
                    if cluster_bi_mean < max_int:
                        max_int = cluster_bi_mean
                
                bi.append(max_int)

    #calculate si
    for x in range(len(ai)):
        if ai[x] == bi[x] or ai[x] == 0:
            si_value = 0
            si.append(si_value)
        else:
            si_value = (bi[x] - ai[x]) / max(bi[x],ai[x])
            si.append(si_value)
    
    #calculate mean si for a cluster
    #mean_si = sum(si) / len(si)
    mean_si = np.mean(si)

    return mean_si

In [8]:
'''

K-means function

Functionality of K-means clustering:
Step 1: Based on the pre-defined number of clusters initialise random centroids using Canopy method 
Step 2: Assign the datapoints to different clusters by finding the minimum distance between the point and centroid
Step 3: Based on the assignment, update the centroids list with the mean of the data points in each cluster
Step 4: Iterate assign and update methods till the centroids dont change. This defines the convergence of K-means algorithm
Step 5: Calculate Silhouette Coeffcient for every cluster to find the optimum cluster for a dataset

Return value: Dataframe and dictionary with points assigned to their respective clusters and SC values for predefined number of clusters

'''

def k_means(dataset,features,data,clusters):
    
    initial_centroid = []   
    i = 1
    distances = 0
    centroids = []
    #sil_arr = []
    #max_si = 0
    sc = []
    
    for clu in clusters:

        #call the canopy function
        centroids = canopy(data,features,clu)
        
        #find the best centroid by calling kmeans_assign and kmeans_update functions iteratively
        prev_centroid = [None] * len(centroids)
        cur_centroid = centroids

        while True:

            data_dict,dataset,final_dataset = kmeans_assign(cur_centroid,data,dataset,features)
            #make a copy of current centroid to previous centroid
            prev_centroid = copy.deepcopy(cur_centroid)
            #update current centroid from the k_means update function
            cur_centroid = kmeans_update(data_dict,features,cur_centroid)
            #find the error between prev_centroid and cur_centroid
            error = (np.sum(prev_centroid,axis = 0) - np.sum(cur_centroid,axis = 0))

            if error.all() == 0:
                break
            else:
                continue

        #call the kmeans_assign to complete the final assignment with best centroids
        data_dict,final_data_pts,final_dataset = kmeans_assign(cur_centroid,data,dataset,features)
        
        #call silhoutte method
        #mean si value returned from silhouette
        sil = silhoutte(data_dict,features)
        #print(sil)
        #sil = np.asscalar(sil)

        #intialise initial_centroid and i back to original values for next cluster iteration
        initial_centroid = []
        i = 1

        #check for cluster number, if k = 1, append sil = 0 to SC
        if clu == 1:
            sc.append(0)
        else:
            sc.append(sil)

    return data_dict,final_dataset,sc

In [10]:
'''

Test for the data validation

The dataset should contain only real numbers and is tested for empty values, alphanumeric and special characters

'''

if dataset.isnull().values.any() or ~dataset.applymap(np.isreal).all().all():
    
    exit()

else:
    
    #create a list for the range of clusters
    cluster_range = [2**n for n in range(0,6)]
    
    '''
    call the k-means function
    kmeans_df = Is the final dataframe with datapoints assigned to their clusters
    sil_coeff = array of Silhouette coefficient for each cluster
    '''
    
    kmeans_dict,kmeans_df,sc = k_means(dataset,features,data,cluster_range)

    #to find optimum clusters,k
    min_sc = 0
    max_sc = float('inf')
    new_k = 0

    #find the cluster number where the sc values first decreases
    for sc_val in range(1,len(sc)-1):
        if sc[sc_val + 1] > sc[sc_val]:
            continue        
        else:
            new_k = sc_val + 1
            break

    #create a new clusters range to find optimum k
    #if cluster_range[new_k] == 2
    new_clu = [clus for clus in range(int(cluster_range[new_k] / (2**2)),cluster_range[new_k]+1)]

    k_means_dict,final_data_pts,sil_coef = k_means(dataset,features,data,new_clu)

    max_sc = max(sil_coef)

    #get the optimum k
    optimum_k = [new_clu[sil_coef.index(max_sc)]]
    #perform k_means on optimum k
    final_dict,final_df,sil_coeff = k_means(dataset,features,data,optimum_k)

    '''

    Anomaly Detection
    
    1. The small clusters with less than a threshold: A dictionary with clusters as keys and count of number of points as values is created. A threshold of 1% of the dataset  is
       calculated and based on the value count in the dictionary, anomalies are detected.
    2. Isolation data points not belong to any cluster: Based on above created dictionary, the clusters with values as 1 as detected as isolation points.
    3. A data point belongs to a cluster with more than 2 standard deviations: Mean, SD and Mean ± 2*SD for each dimension is calculated at cluster level and points not in this
       range as detected as anomalies.

    '''

    #create an anomaly array
    anomaly_array = []
    
    threshold = int(0.01 * len(data))

    #create array of final dataframe
    final_arr = final_df.values

    #dictionary with cluster and count of number of points
    cluster_len = {}

    for s in range(len(final_arr)):
        if final_arr[s][-1] in cluster_len:
            cluster_len[final_arr[s][-1]] += 1
        else:
            cluster_len[final_arr[s][-1]] = 1

    for clster,len_val in cluster_len.items():
        #detect isolation points
        if len_val == 1:
            anomaly_array.append(final_dict[clster])
        #detect clusters with points less than threshold
        elif len_val < threshold:
            for values in final_dict[clster]:
                anomaly_array.append(values)

    anamoly_dict = {}
    for clu_key,clu_val in final_dict.items():
        anamoly_dict[clu_key] = [(np.mean(clu_val,axis = 0) + 2*np.std(clu_val,axis = 0)).tolist(),(np.mean(clu_val,axis = 0) - 2*np.std(clu_val,axis = 0)).tolist()]
        
    z = 0
    for key1 in set(anamoly_dict.keys()) & set(final_dict.keys()):
        for value1 in final_dict[key1]:
            #check if each point is within the mean ± 2*SD range
            if ~(np.less(np.array(value1),np.array(anamoly_dict[key1][z])).any()) and np.greater(np.array(np.array(value1)),anamoly_dict[key1][z+1]).all():
                anomaly_array.append(value1)
            else:
                continue

    anomalies_array = [[np.round(float(i), 2) for i in nested] for nested in anomaly_array]
    
    if len(anomalies_array) == 0:
        print("No anomalies")
    else:
        for anomaly in anomalies_array:
            print(anomaly)

[-771.2, -1582.8]
[-1582.0, -1582.7]
